In [1]:
import spektral.data as data
import tensorflow as tf
import numpy as np 
from tensorflow import keras
import os
import sys
import argparse
import math
from math import floor
import pandas as pd 
import pickle

from GraphDataset import GraphDataset
from autoencoder import EncoderGAE,EncoderVGAE, DecoderX, DecoderA, VGAE, GAE
from utils import *
from custom_layers import *
from custom_layers import ConvTypes
from LossManager import LossManager, LossTypes
from framasToGraph import FramsTransformer
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

def ensureDir(string):
    if os.path.isdir(string):
        return string
    else:
        raise NotADirectoryError(string)
        
def get_convType(name):
    if name =="gcnconv":
        return ConvTypes.GCNConv
    elif name =="armaconv":
        return ConvTypes.ARMAConv
    elif name =="eccconv":
        return ConvTypes.ECCConv
    elif name =="gatconv":
        return ConvTypes.GATConv
    elif name =="gcsconv":
        return ConvTypes.GCSConv
    
def get_Loss(name):
    if name =="joints":
        return LossTypes.joints
    elif name =="parts":
        return LossTypes.parts
    elif name =="fitness":
        return LossTypes.fitness
    elif name =="dissim":
        return LossTypes.dissim
    elif name =="None":
        return LossTypes.No
    elif name == "No":
        return LossTypes.No


def load_config(pathconfig):
    with open(pathconfig) as file:
        lines = file.readlines()
        lines = [line.rstrip() for line in lines]
    params_dict = {} 
    params_dict['pathframs']="/home/adam/Framsticks/Framsticks50rc19"
    params_dict['pathdata']="graphs/gen/"
    params_dict['pathout']="experiments/experiment_base/models/"
    params_dict['batchsize']=int(lines[3])
    params_dict['adjsize']=int(lines[4])
    params_dict['numfeatures']=int(lines[5])
    params_dict['latentdim']=int(lines[6])
    params_dict['nhidden']=int(lines[7])
    params_dict['convenc']=int(lines[8])
    params_dict['denseenc']=int(lines[9])
    params_dict['densedeca']=int(lines[10])
    params_dict['convdecx']=int(lines[11])
    params_dict['densedecx']=int(lines[12])
    params_dict['learningrate']=float(lines[13])
    params_dict['epochs']=int(lines[14])
    params_dict['convtype']=get_convType(lines[15])
    params_dict['variational']=lines[16]
    params_dict['loss']=get_Loss(lines[17])
    params_dict['trainid']=lines[18]
    return params_dict



In [13]:
             
class AE_evolalg:
    def __init__(self,path_config) -> None:
        self.params_dict = load_config(path_config)

        self.frams_manager = FramsManager(self.params_dict['pathframs'])
        self.frams_transformer = FramsTransformer(self.params_dict['pathframs'],self.params_dict['adjsize'])
        ae_type=None
        if self.params_dict['variational'] == "True":
            variational=True
        else:
            variational=False

        if variational == True:
            ae_type="VGAE"
        else:
            ae_type="GAE"

        self.path_out = (str(self.params_dict['pathout'])+
                    str(self.params_dict['loss'])+
                    "/"+ae_type +
                    "/numfeatures"+str(self.params_dict['numfeatures']) +
                    "/adjsize"+str(self.params_dict['adjsize']) + 
                    "/batchsize"+str(self.params_dict['batchsize']) +
                    "/latentdim"+str(self.params_dict['latentdim'])+
                    "/nhidden"+str(self.params_dict['nhidden'])+
                    "/learningrate"+str(self.params_dict['learningrate'])+
                    "/convtype"+str(self.params_dict['convtype'])+
                    "/"
                    )
        self.model_name = ("model_enc_"+str(self.params_dict['convenc'])+"_"+str(self.params_dict['denseenc'])+
                    "_deca"+str(self.params_dict['densedeca'])+
                    "_decx"+str(self.params_dict['convdecx'])+"_"+str(self.params_dict['densedecx'])+
                    "_train_id_"+str(self.params_dict['trainid'])
                    )
                    
        if self.params_dict['loss'] is not LossTypes.No:
            lossManager = LossManager(self.params_dict['pathframs'],"eval-allcriteria.sim","vertpos")
            if self.params_dict['loss'] == LossTypes.joints:
                custom_loss = lossManager.joints_too_big_loss
            elif self.params_dict['loss'] == LossTypes.parts:
                custom_loss = lossManager.part_number_loss
            elif self.params_dict['loss'] == LossTypes.fitness:
                custom_loss = lossManager.fitness_comparison_loss
            elif self.params_dict['loss'] == LossTypes.dissim:
                custom_loss = lossManager.dissimilarity_comparison
            else:
                print(self.params_dict['loss']," is not supported, custom loss set to None")
                custom_loss = None
        else:
            custom_loss = None

        train, test = GraphDataset(self.params_dict['pathframs'], self.params_dict['pathdata'],size_of_adj=self.params_dict['adjsize']).read()

        loader_train = data.BatchLoader(train, batch_size=self.params_dict['batchsize'])
        loader_test = data.BatchLoader(test, batch_size=self.params_dict['batchsize'])

        if variational == True:
            encoder = EncoderVGAE(latent_dim=self.params_dict['latentdim'],
                            n_hidden=self.params_dict['nhidden'],
                            num_conv=self.params_dict['convenc'],
                            num_dense=self.params_dict['denseenc'],
                            convtype=self.params_dict['convtype'])
        else:
            encoder = EncoderGAE(latent_dim=self.params_dict['latentdim'],
                        n_hidden=self.params_dict['nhidden'],
                        num_conv=self.params_dict['convenc'],
                        num_dense=self.params_dict['denseenc'],
                        convtype=self.params_dict['convtype'])
                            
        decoderA = DecoderA(adjency_size=self.params_dict['adjsize'],
                            latent_dim=self.params_dict['latentdim'],
                            num_dense=self.params_dict['densedeca'])
        decoderX = DecoderX(latent_dim=self.params_dict['latentdim'], 
                            adjency_size=self.params_dict['adjsize'],
                            num_features=self.params_dict['numfeatures'],
                            num_conv=self.params_dict['convdecx'],
                            num_dense=self.params_dict['densedecx'],
                            convtype=self.params_dict['convtype'])


        current_learning_rate = self.params_dict['learningrate']

        if variational == True:
            self.autoencoder = VGAE(encoder,decoderA,decoderX,custom_loss)
        else:
            self.autoencoder = GAE(encoder,decoderA,decoderX,custom_loss)

        opt = keras.optimizers.Adam(learning_rate=current_learning_rate)
        self.autoencoder.compile(optimizer=opt)

        x,a = next(loader_train)
        _ = self.autoencoder.train_step([(tf.convert_to_tensor(x)),
                                            tf.convert_to_tensor(a)])
        self.autoencoder.built = True

        if os.path.exists(self.path_out+self.model_name):
            print("Trying to load the model")
            _, _  = load_model(self.path_out,self.model_name,self.autoencoder)

        else:
            os.makedirs(self.path_out+self.model_name)

        self.path_save = self.path_out+self.model_name

    def prepareGenList(self,X,A):
        gen_list = gen_f0_from_tensors(X,A)
        gen_correct = []
        for g in gen_list:      
            gen = self.frams_manager.check_consistency_for_gen(g)
            if gen is not None:
                gen = self.frams_manager.reduce_joint_length_for_gen(gen)
            if gen is not None:
                gen_correct.append(gen)
        return gen_correct

    def prepareXAforPopulation(self,population):
        x_all = []
        a_all = []
        for p in population:
            graph = self.frams_transformer.getGrafFromString(p['genotype']) 
            x_all.append(graph.x)
            a_all.append(graph.a)
        return x_all,a_all

    def mutate_population(self,population):
        # translate to a and x
        x,a = self.prepareXAforPopulation(population)
        orginal_z  = self.autoencoder.encoder([(tf.convert_to_tensor(x)),
                                      tf.convert_to_tensor(a)])
        # mutate by adding random stuff to z 
        new_z_list = []
        for z in orginal_z:
            mutation  = np.random.uniform(-1,1,self.params_dict['latentdim'])
            m_z = z + mutation
            new_z_list.append(m_z)
        new_z = tf.convert_to_tensor(new_z_list)
        recA = self.autoencoder.decoderA(new_z)
        recX = self.autoencoder.decoderX([new_z,recA])

        geno = self.prepareGenList(recX,recA)
        return geno 




In [14]:
population=[{"genotype":"XXXX"},{"genotype":"XXXX"},{"genotype":"XXXX"}]

In [15]:
path = '/home/adam/thesis/VGAE/experiments/experiment_base/models/No/GAE/numfeatures3/adjsize15/batchsize256/latentdim15/nhidden64/learningrate0.01/convtypeGCSConv/model_enc_2_2_deca1_decx1_2_train_id_9/args.txt'

In [16]:
autoencoder = AE_evolalg(path)
autoencoder.mutate_population(population)

Using Framsticks version: 5.0rc19.1
Home (writable) dir     : /home/adam/Framsticks/Framsticks50rc19/data
Resources dir           : /home/adam/Framsticks/Framsticks50rc19/data

Using Framsticks version: 5.0rc19.1
Home (writable) dir     : /home/adam/Framsticks/Framsticks50rc19/data
Resources dir           : /home/adam/Framsticks/Framsticks50rc19/data

Using Framsticks version: 5.0rc19.1
Home (writable) dir     : /home/adam/Framsticks/Framsticks50rc19/data
Resources dir           : /home/adam/Framsticks/Framsticks50rc19/data

[Errno 2] No such file or directory: 'graphs/gen/142-checkpoint.gen'
pass
[Errno 2] No such file or directory: 'graphs/gen/57-checkpoint.gen'
pass
[Errno 2] No such file or directory: 'graphs/gen/0-checkpoint.gen'
pass
[Errno 2] No such file or directory: 'graphs/gen/145-checkpoint.gen'
pass
[Errno 2] No such file or directory: 'graphs/gen/250-checkpoint.gen'
pass
[Errno 2] No such file or directory: 'graphs/gen/195-checkpoint.gen'
pass

If you intended to run this

/home/adam/.local/lib/python3.7/site-packages/spektral/data/utils.py:198: UserWarning: `x` isn't a recognized object; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  shuffle_inplace(*data)


Trying to load the model
Model loaded successfully
Losses loaded successfully

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



['//0\np:0.39464855, 0.21630478, 0.48555756\np:0.57157326, 0.5239067, 0.4201355\np:0.96445656, 1.2205954, 0.6864662\np:0.9822836, 1.229126, 0.5600605\np:1.8365755, 0.9017973, 0.12487221\np:2.6917095, 0.7064729, -1.0\np:0.35708284, 0.81672645, 0.5417242\np:1.8695095, -1.0, 1.5518997\nj:0, 1\nj:1, 2\nj:2, 3\nj:3, 4\nj:4, 5\n',
 '//0\np:12.503392168786196, 3.3901964110437635, 5.746707235586162\np:10.507595, 3.517971, 5.768232\np:8.543174037672722, 3.825408538490246, 5.983936526371684\np:10.184617554344527, 3.079675409667026, 5.11817600396804\np:8.474954955138983, 3.589630226996586, 6.022054248623326\np:10.161146603444823, 3.0543582533600198, 5.089188963909693\np:2.1952171, -1.0, -1.0\np:5.111048, -1.0, -1.0\nj:0, 1\nj:1, 2\nj:2, 3\nj:3, 4\nj:4, 5\n']

In [18]:
print('//0\np:4.3582573, 5.96474, 4.9959607\np:3.8931093, 5.600706, 4.9703\np:3.9434462, 5.673972, 5.381123\np:5.798353611935842, 6.161602332596109, 5.205747267806096\np:1.6317005, 3.6080866, 3.1242266\np:4.4167356, -1.0, -1.0\nj:0, 1\nj:1, 2\nj:1, 3\n')

//0
p:4.3582573, 5.96474, 4.9959607
p:3.8931093, 5.600706, 4.9703
p:3.9434462, 5.673972, 5.381123
p:5.798353611935842, 6.161602332596109, 5.205747267806096
p:1.6317005, 3.6080866, 3.1242266
p:4.4167356, -1.0, -1.0
j:0, 1
j:1, 2
j:1, 3

